In [6]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Define the VGGNet architecture
class VGGNet(nn.Module):
    def __init__(self, num_classes=10):
        super(VGGNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1) 
        x = self.classifier(x)
        return x

# Create an instance of the VGGNet model
vggnet = VGGNet()


print(vggnet)



VGGNet(
  (features): Sequential(
    (0): Conv2d(1, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [7]:
batch_size = 64
learning_rate = 0.001
epochs = 10

# Data preprocessing and loading
transform = transforms.Compose([transforms.Resize((32, 32)),  # Resize MNIST images to (32, 32) to match VGGNet input size
                                transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


vggnet = VGGNet()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vggnet.parameters(), lr=learning_rate, momentum=0.9)

# Training loop
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        # Forward pass
        outputs = vggnet(inputs)
        loss = criterion(outputs, labels)
        
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  
            print(f'Epoch {epoch + 1}, Mini-batch {i + 1}, Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Finished Training')


vggnet.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = vggnet(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Epoch 1, Mini-batch 100, Loss: 2.3029
Epoch 1, Mini-batch 200, Loss: 2.3016
Epoch 1, Mini-batch 300, Loss: 2.3023
Epoch 1, Mini-batch 400, Loss: 2.3026
Epoch 1, Mini-batch 500, Loss: 2.3021
Epoch 1, Mini-batch 600, Loss: 2.3007
Epoch 1, Mini-batch 700, Loss: 2.3013
Epoch 1, Mini-batch 800, Loss: 2.3013
Epoch 1, Mini-batch 900, Loss: 2.3002
Epoch 2, Mini-batch 100, Loss: 2.3017
Epoch 2, Mini-batch 200, Loss: 2.3012
Epoch 2, Mini-batch 300, Loss: 2.3024
Epoch 2, Mini-batch 400, Loss: 2.3029
Epoch 2, Mini-batch 500, Loss: 2.3010
Epoch 2, Mini-batch 600, Loss: 2.3020
Epoch 2, Mini-batch 700, Loss: 2.3006
Epoch 2, Mini-batch 800, Loss: 2.3025
Epoch 2, Mini-batch 900, Loss: 2.3022
Epoch 3, Mini-batch 100, Loss: 2.3021
Epoch 3, Mini-batch 200, Loss: 2.3015
Epoch 3, Mini-batch 300, Loss: 2.3005
Epoch 3, Mini-batch 400, Loss: 2.3009
Epoch 3, Mini-batch 500, Loss: 2.3022
Epoch 3, Mini-batch 600, Loss: 2.3006
Epoch 3, Mini-batch 700, Loss: 2.3014
Epoch 3, Mini-batch 800, Loss: 2.3010
Epoch 3, Min